In [ ]:
!uv pip install -U Ipython
!uv pip install webvtt-py sacrebleu rouge_score pycocoevalcap
!uv pip install peft hf_xet hf-trim evaluate
!pip install git+https://github.com/google-research/bleurt.git

In [2]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/MyDrive/gfslt-pose-trainer
%load_ext autoreload
%autoreload 2

# !wget https://storage.googleapis.com/bleurt-oss-21/BLEURT-20.zip -O ./checkpoints/BLEURT-20.zip
# !git fetch origin && git reset --hard origin/main # Don't care about local changes and overwrite everything
# !git pull
!ls

Mounted at /content/drive/
/content/drive/MyDrive/gfslt-pose-trainer
backbones    __init__.py  __pycache__	    stage2.py	     utils.py
checkpoints  loader.py	  README.md	    trim_mbart.py
config.py    models.py	  requirements.txt  trimmed_mbart
dataset      poses	  stage1.py	    Untitled0.ipynb


In [ ]:
# Take around 14-16mins on A100, 30mins on T4
!rm -rf /tmp/auto_sat_aligned
!rm -rf /tmp/manual_sat_aligned
!rm -rf /tmp/bobsl_dwpose
!7z x ./dataset/auto_sat_aligned.zip -o/tmp
!7z x ./dataset/manual_sat_aligned.zip -o/tmp
!7z x ./dataset/BOBSL/bobsl_dwpose.zip -o/tmp
!7z x ./checkpoints/BLEURT-20.zip -o/tmp
# !python3 trim_mbart.py # Prepare trimmed MBart tokenizer/model

In [18]:
%%time
!python3 stage1.py \
	--output_dir /tmp/gfslt_stage1 \
	--max_event_tokens 40 \
	--embed_dim 1024 \
	--num_train_epochs 80 \
	--learning_rate 0.01 \
    --weight_decay 0.0 \
	--per_device_train_batch_size 128

2026-01-09 12:31:16.290415: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-09 12:31:16.309351: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767961876.331210   26542 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767961876.337912   26542 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767961876.354666   26542 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [25]:
%%time
!python3 stage2.py \
	--output_dir /tmp/gfslt_stage2 \
    --stage1_checkpoint ./checkpoints/gfslt_stage1/pytorch_model.bin \
	--max_event_tokens 40 \
	--embed_dim 1024 \
	--num_train_epochs 200 \
	--learning_rate 0.001 \
    --weight_decay 1e-3 \
	--per_device_train_batch_size 180 \
	--do_train

2026-01-09 13:29:07.570632: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-09 13:29:07.591330: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767965347.614245   47032 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767965347.621540   47032 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767965347.639801   47032 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [32]:
%%time
!python3 stage2.py \
	--output_dir /tmp/gfslt_stage2 \
	--max_event_tokens 40 \
	--embed_dim 1024 \
	--num_train_epochs 200 \
	--learning_rate 0.001 \
    --weight_decay 1e-3 \
	--per_device_eval_batch_size 1024 \
	--do_eval

2026-01-09 15:46:59.448398: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-09 15:46:59.469712: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767973619.493213   95293 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767973619.500313   95293 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767973619.519031   95293 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 